In [1]:
# imports and setup 

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, KFold

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout


Using TensorFlow backend.


In [2]:
def organize_pixel_values(raw_df):
    df = raw_df.copy();
    for i in range(12):
        df.drop(df.columns[[0]], axis=1, inplace=True)

    ret = []
    column_vals = df.columns
    df_matrix = df.to_numpy()
    
    for row in range(len(df_matrix)):
        pixel_matrix = np.zeros((20, 20))
        for col in range(len(df_matrix[0])):
            header = column_vals[col]
            splitHeader = header[1:].split('c')
            pixelRow = splitHeader[0]
            pixelCol = splitHeader[1]
            pixel_matrix[int(pixelRow)][int(pixelCol)] = df_matrix[row][col] / 255.0
        ret.append(pixel_matrix)
    
    ret = np.stack(ret)
    ret = np.reshape(x, (-1, 20, 20, 1))
    
    return ret

In [ ]:
def get_maps_from_labels(input_arr):  
    val_to_ix = { val:i for i,val in enumerate(np.unique(y_raw)) }
    ix_to_val = { i:val for i,val in enumerate(np.unique(y_raw)) }
    
    return val_to_ix, ix_to_val

In [46]:
class font:
    df
    x
    y
    unique_char_count
    val_to_ix
    val_to_char
    
    def __init__(self, file_path):
        self.df = pd.read_csv('fonts/ARIAL.csv')
        y_raw = np.array(df.array)
        self.val_to_ix, self.val_to_char = get_maps_from_lables(y_raw)

In [47]:
arial = font('fonts/ARIAL.csv')

In [4]:
y_raw = np.array(df.m_label)
x = organize_pixel_values(df)


# print(len(y_raw))
# print(len(x))
# print(len(x[0]))
# print(len(x[0][0]))
# print(x[0])

In [6]:
val_to_ix, val_to_char = get_maps_from_labels(y_raw)

# print(ix_to_char[32])
# print(char_to_ix[64])

In [7]:
unique_char_count = len(val_to_ix)

y = []
for val in y_raw:
    arr = np.zeros(unique_char_count)
    arr[val_to_ix[val]] = 1;
    y.append(arr)

y = np.array(y)

# print(unique_char_count)
# print(len(y_raw))
# print(len(y[0]))
# print(len(y))

In [8]:
model = Sequential()

model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model.add(MaxPooling2D((2, 2), padding="same"))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D((2, 2), padding="same"))
model.add(Flatten())
model.add(Dropout(.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(unique_char_count, activation='softmax'))

W0710 14:51:14.895457 4669994432 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0710 14:51:14.909225 4669994432 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0710 14:51:14.911177 4669994432 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0710 14:51:14.923674 4669994432 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0710 14:51:14.943591 4669994432 deprecation_wrapper.py:119] From /usr/loca

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

W0710 14:51:14.984170 4669994432 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0710 14:51:15.005231 4669994432 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [39]:
x_train, x_test, y_train, y_test = train_test_split(x_reshaped, y, random_state=1, test_size=0.8)

print(x_reshaped.shape)

(26237, 20, 20, 1)


In [20]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

Train on 5247 samples, validate on 20990 samples
Epoch 1/20
5247/5247 [==============================] - 5s 992us/step - loss: 3.4045 - acc: 0.3848 - val_loss: 5.5738 - val_acc: 0.3579
Epoch 2/20
5247/5247 [==============================] - 5s 924us/step - loss: 2.5925 - acc: 0.4513 - val_loss: 5.5182 - val_acc: 0.3870
Epoch 3/20
5247/5247 [==============================] - 5s 932us/step - loss: 2.0328 - acc: 0.5079 - val_loss: 5.3663 - val_acc: 0.4065
Epoch 4/20
5247/5247 [==============================] - 6s 1ms/step - loss: 1.6738 - acc: 0.5573 - val_loss: 5.3792 - val_acc: 0.4177
Epoch 5/20
5247/5247 [==============================] - 5s 912us/step - loss: 1.4120 - acc: 0.6003 - val_loss: 5.3945 - val_acc: 0.4240
Epoch 6/20
5247/5247 [==============================] - 4s 851us/step - loss: 1.2449 - acc: 0.6371 - val_loss: 5.3341 - val_acc: 0.4316
Epoch 7/20
5247/5247 [==============================] - 5s 867us/step - loss: 1.1088 - acc: 0.6745 - val_loss: 5.3448 - val_acc: 0.4398
E

### Evaluate the network using cross validation (splitting data into training/testing). What is its accuracy? ###

**Training and testing on Arial, our model gives us an accuracy of around 82% after 20 epochs**

In [22]:
model2 = Sequential()

model2.add(Conv2D(128, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model2.add(MaxPooling2D((2, 2), padding="same"))
model2.add(Conv2D(128, kernel_size=3, activation='relu'))
model2.add(MaxPooling2D((2, 2), padding="same"))
model2.add(Flatten())
model2.add(Dropout(.1))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(unique_char_count, activation='softmax'))

In [23]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

Train on 5247 samples, validate on 20990 samples
Epoch 1/20
5247/5247 [==============================] - 11s 2ms/step - loss: 6.4465 - acc: 0.1896 - val_loss: 5.7674 - val_acc: 0.2899
Epoch 2/20
5247/5247 [==============================] - 12s 2ms/step - loss: 4.8165 - acc: 0.3112 - val_loss: 5.3951 - val_acc: 0.3081
Epoch 3/20
5247/5247 [==============================] - 12s 2ms/step - loss: 3.6847 - acc: 0.3751 - val_loss: 5.4963 - val_acc: 0.3708
Epoch 4/20
5247/5247 [==============================] - 12s 2ms/step - loss: 2.6442 - acc: 0.4667 - val_loss: 5.2158 - val_acc: 0.4047
Epoch 5/20
5247/5247 [==============================] - 12s 2ms/step - loss: 1.9120 - acc: 0.5333 - val_loss: 5.2117 - val_acc: 0.4180
Epoch 6/20
5247/5247 [==============================] - 13s 2ms/step - loss: 1.4833 - acc: 0.5939 - val_loss: 5.1709 - val_acc: 0.4340
Epoch 7/20
5247/5247 [==============================] - 12s 2ms/step - loss: 1.2221 - acc: 0.6468 - val_loss: 5.1147 - val_acc: 0.4437
Epoch 

### Create and train a different network topology (add more convolution/dropout layers, explore other types/sizes of layer). Try to find a topology that works better than the one described above. ###

**I took a very simple approach with my second model, I just doubled the number of perceptrons of the convolutional and dense layers. This new model gives us a modest increase of accuracy (84% after 20 epochs).  Each epoch took between 2 and 2.5x as long to run though, so it's possible that the increased accuracy would not be worth the time/computing cost in practice**

In [28]:
model3 = Sequential()

model3.add(Conv2D(64, kernel_size=5, activation='relu', input_shape=(20, 20, 1)))
model3.add(MaxPooling2D((2, 2), padding="same"))
model3.add(Conv2D(64, kernel_size=5, activation='relu'))
model3.add(MaxPooling2D((2, 2), padding="same"))
model3.add(Flatten())
model3.add(Dropout(.1))
model3.add(Dense(64, activation='relu'))
model3.add(Dense(unique_char_count, activation='softmax'))

In [30]:
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
model3.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

Train on 5247 samples, validate on 20990 samples
Epoch 1/20
5247/5247 [==============================] - 6s 1ms/step - loss: 6.6763 - acc: 0.1498 - val_loss: 5.9241 - val_acc: 0.2391
Epoch 2/20
5247/5247 [==============================] - 5s 972us/step - loss: 5.0699 - acc: 0.2914 - val_loss: 5.5811 - val_acc: 0.2917
Epoch 3/20
5247/5247 [==============================] - 5s 941us/step - loss: 4.1209 - acc: 0.3276 - val_loss: 5.5674 - val_acc: 0.3326
Epoch 4/20
5247/5247 [==============================] - 5s 887us/step - loss: 3.2560 - acc: 0.4006 - val_loss: 5.4225 - val_acc: 0.3732
Epoch 5/20
5247/5247 [==============================] - 5s 880us/step - loss: 2.5727 - acc: 0.4490 - val_loss: 5.3024 - val_acc: 0.3965
Epoch 6/20
5247/5247 [==============================] - 5s 873us/step - loss: 2.0823 - acc: 0.4961 - val_loss: 5.1964 - val_acc: 0.4136
Epoch 7/20
5247/5247 [==============================] - 5s 879us/step - loss: 1.7761 - acc: 0.5312 - val_loss: 5.1609 - val_acc: 0.4232
E

In [36]:
model4 = Sequential()

model4.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(20, 20, 1)))
model4.add(MaxPooling2D((2, 2), padding="same"))
model4.add(Conv2D(64, kernel_size=3, activation='relu'))
model4.add(MaxPooling2D((2, 2), padding="same"))
model4.add(Conv2D(64, kernel_size=3, activation='relu'))
model4.add(MaxPooling2D((2, 2), padding="same"))
model4.add(Flatten())
model4.add(Dropout(.1))
model4.add(Dense(64, activation='relu'))
model4.add(Dense(unique_char_count, activation='softmax'))

In [37]:
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
model4.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

Train on 5247 samples, validate on 20990 samples
Epoch 1/20
5247/5247 [==============================] - 7s 1ms/step - loss: 6.9158 - acc: 0.0667 - val_loss: 6.3631 - val_acc: 0.1815
Epoch 2/20
5247/5247 [==============================] - 6s 1ms/step - loss: 5.6002 - acc: 0.2424 - val_loss: 5.7579 - val_acc: 0.2681
Epoch 3/20
5247/5247 [==============================] - 7s 1ms/step - loss: 4.7324 - acc: 0.2977 - val_loss: 5.5887 - val_acc: 0.2925
Epoch 4/20
5247/5247 [==============================] - 7s 1ms/step - loss: 4.0835 - acc: 0.3219 - val_loss: 5.5692 - val_acc: 0.3213
Epoch 5/20
5247/5247 [==============================] - 7s 1ms/step - loss: 3.5417 - acc: 0.3570 - val_loss: 5.6112 - val_acc: 0.3432
Epoch 6/20
5247/5247 [==============================] - 6s 1ms/step - loss: 3.0736 - acc: 0.3896 - val_loss: 5.5713 - val_acc: 0.3685
Epoch 7/20
5247/5247 [==============================] - 6s 1ms/step - loss: 2.6713 - acc: 0.4187 - val_loss: 5.5341 - val_acc: 0.3766
Epoch 8/20
52

### Third and Fourth Models ###

**I did a third and forth model mostly for my enjoyment (since the second model was already an improvment over the first).  The third model I changed the kernel size to 5x5 and it underpreformed with 75% accuracy.  The fourth model I added one additional convolutional layer * max pooling layer and (to my surprise) it dramatically underperformed at 64% accuracy**

In [ ]:
df2 = pd.read_csv('fonts/TIMES.csv')

In [ ]:
y_raw2 = np.array(df2.m_label)
x2 = organize_pixel_values(df2)

In [ ]:
val_to_ix2, val_to_char2 = get_maps_from_labels(y_raw2)

In [ ]:
unique_char_count2 = len(val_to_ix)

y2 = []
for val in y_raw2:
    arr = np.zeros(unique_char_count)
    arr[val_to_ix[val]] = 1;
    y.append(arr)

y = np.array(y)

# print(unique_char_count)
# print(len(y_raw))
# print(len(y[0]))
# print(len(y))


Create and train a different network topology (add more convolution/dropout layers, explore other types/sizes of layer). Try to find a topology that works better than the one described above.


Test the accuracy of your network with character inputs from a DIFFERENT font set. How does it perform? (He means train on one font, test on another)

Train your best network on inputs from the data from at least 2 different fonts. How does your accuracy compare to the 1-font case? What accuracy do you see when testing with inputs from a font you didn't train on?
Take a look at some of the characters that have been misclassified. Do you notice any patterns? The network only produces the relative probabilities that the input is any of the possible characters. Can you find examples where the network is unsure of the result?